# Python for Excel

In [ ]:
from pathlib import Path
import pandas as pd

# Having a quick look

In [ ]:
# requires "sales_data" folder from
# https://github.com/fzumstein/python-for-excel
df = pd.read_excel('sales_data/new/January.xlsx')
df

In [ ]:
df.info()

In [ ]:
df.describe()

# Reading all files into a pandas DataFrame

In [ ]:
# Read in all Excel files from all subfolders of sales_data
parts = []
for path in Path("sales_data").rglob("*.xls*"):
    print(f'Reading {path.name}')
    part = pd.read_excel(path, index_col="transaction_id")
    parts.append(part)

In [ ]:
# Combine the DataFrames from each file into a single DataFrame
# pandas takes care of properly aligning the columns
data = pd.concat(parts)

In [ ]:
data.sort_values('transaction_date')

In [ ]:
data.info()

# Pivot Table

In [ ]:
# Pivot each store into a column and sum up all transactions per date
pivot = pd.pivot_table(data,
                       index="transaction_date", columns="store",
                       values="amount", aggfunc="sum")
pivot

# Resampling: Aggregating by month

In [ ]:
# Resample to end of month and assign an index name
summary = pivot.resample("M").sum()
summary.index.name = "Month"
summary

# Plotting with Plotly

In [ ]:
pd.options.plotting.backend = "plotly"
summary.plot.bar(barmode="group")

# Writing an Excel report with pandas

In [ ]:
# Write summary report to Excel file
summary.to_excel("sales_report_pandas.xlsx")

# Excel Automation with xlwings

## Interactive use

In [ ]:
import xlwings as xw
from xlwings import view, load

In [ ]:
view(summary)

In [ ]:
df = load()
df

## True editing of Excel files

In [ ]:
template = xw.Book("sales_report_template.xlsx")
sheet = template.sheets["Sheet1"]

In [ ]:
sheet["B3"].value = summary

In [ ]:
sheet["B3"].expand().columns.autofit()

In [ ]:
sheet.charts["Chart 1"].set_source_data(sheet["B3"].expand())

In [ ]:
template.save("sales_report_xlwings.xlsx")